In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import numpy as np
import torchvision.transforms as transforms
import os
import sys
#from models import *
sys.path.append("../..")

In [2]:
import torch.nn as nn
class SEWeightModule(nn.Module):
    def __init__(self, channels, reduction=4):
        super(SEWeightModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels//reduction, kernel_size=1, padding=0)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(channels//reduction, channels, kernel_size=1, padding=0)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.avg_pool(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        weight = self.sigmoid(out)

        return weight

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F


class RB1(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(RB1, self).__init__()
        aa=2
        self.conv1 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=1,dilation=1)
        self.bn1 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv2 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=2,dilation=2)
        self.bn2 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv3 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=3,dilation=3)
        self.bn3 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv4 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=4,dilation=4)
        self.bn4 = nn.BatchNorm2d(out_channels//aa)        
        
        self.conv6 = nn.Conv2d(out_channels*4//aa, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(out_channels)  

        self.conv7 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn7 = nn.BatchNorm2d(out_channels)          
        
        #self.Drop =torch.nn.Dropout(p=0.2, inplace=False)
        self.p = nn.PReLU()
        self.se = SEWeightModule(out_channels*2,2)

    def forward(self, x):
        output1 = self.conv1(x)
        output1 = self.p(self.bn1(output1))
        
        output2 = self.conv2(x)
        output2 = self.p(self.bn2(output2))       
        
        output3 = self.conv3(x)
        output3 = self.p(self.bn3(output3))     
        
        output4 = self.conv4(x)
        output4 = self.p(self.bn4(output4))
        
        output = torch.cat([output1,output2,output3,output4], 1)
        Woutput= self.se(output)
        output = output * Woutput
        
        xd = self.conv7(x)
        xd = self.p(self.bn7(xd))  
        
        #print((output+xd).shape)
        
        #output6 = self.conv6( output+xd )
        output6 = self.conv6(output) 
        output6 = self.p(self.bn6(output6)+xd )

        return output6 
                         
import torch
import torch.nn as nn
from torch.nn import functional as F


class RB2(nn.Module):
    def __init__(self, in_channels, out_channels):

        super(RB2, self).__init__()
        aa=2
        self.conv1 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=1,dilation=1)
        self.bn1 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv2 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=2,dilation=2)
        self.bn2 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv3 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=3,dilation=3)
        self.bn3 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv4 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=4,dilation=4)
        self.bn4 = nn.BatchNorm2d(out_channels//aa)          
        
        self.conv6 = nn.Conv2d(out_channels*4//aa, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn6 = nn.BatchNorm2d(out_channels)  

        self.conv7 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.bn7 = nn.BatchNorm2d(out_channels)          
        
        #self.Drop =torch.nn.Dropout(p=0.2, inplace=False)
        self.p = nn.PReLU()
        self.se = SEWeightModule(out_channels*2,2)

    def forward(self, x):
        output1 = self.conv1(x)
        output1 = self.p(self.bn1(output1))
        
        output2 = self.conv2(x)
        output2 = self.p(self.bn2(output2))       
        
        output3 = self.conv3(x)
        output3 = self.p(self.bn3(output3))     
        
        output4 = self.conv4(x)
        output4 = self.p(self.bn4(output4))
        
        output = torch.cat([output1,output2,output3,output4], 1)
        Woutput= self.se(output)
        output = output * Woutput
        
        xd = self.conv7( x)
        xd = self.p(self.bn7(xd))  
        
        output6 = self.conv6(output)    
        output6 = self.p(self.bn6(output6)+xd )       

        return output6 
    
import torch
import torch.nn as nn
from torch.nn import functional as F


class RB3(nn.Module):
    def __init__(self, in_channels, out_channels):
        
        super(RB3, self).__init__()
        aa=2
        self.conv1 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=1,dilation=1)
        self.bn1 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv2 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=2,dilation=2)
        self.bn2 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv3 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=3,dilation=3)
        self.bn3 = nn.BatchNorm2d(out_channels//aa)
        
        self.conv4 = nn.Conv2d(in_channels, out_channels//aa, kernel_size=3, stride=1, padding=4,dilation=4)
        self.bn4 = nn.BatchNorm2d(out_channels//aa)             
        
        self.conv6 = nn.Conv2d(out_channels*4//aa, 64, kernel_size=3, stride=1, padding=1, groups=2)
        
        self.bn6 = nn.BatchNorm2d(64)  
        

        self.conv7 = nn.Conv2d(in_channels, 64, kernel_size=1, stride=1, padding=0)
        self.bn7 = nn.BatchNorm2d(64)          
        
        #self.Drop =torch.nn.Dropout(p=0.2, inplace=False)
        self.p = nn.PReLU()
        self.se = SEWeightModule(out_channels*2,2)

    def forward(self, x):
        output1 = self.conv1(x)
        output1 = self.p(self.bn1(output1))
        
        output2 = self.conv2(x)
        output2 = self.p(self.bn2(output2))       
        
        output3 = self.conv3(x)
        output3 = self.p(self.bn3(output3))     
        
        output4 = self.conv4(x)
        output4 = self.p(self.bn4(output4))
        
        output = torch.cat([output1,output2,output3,output4], 1)
        Woutput= self.se(output)
        output = output * Woutput
        
        xd = self.conv7( x)
        xd = self.p(self.bn7(xd))  
        
        output6 = self.conv6(output)    
        output6 = self.p(self.bn6(output6)+xd )      

        return output6 
    
import torch
import torch.nn as nn
from torch.nn import functional as F


class MAPB(nn.Module):
    def __init__(self):
        
        super(MAPB, self).__init__()
        self.conv1 = RB1(1,16)
        self.conv2 = RB2(16,32)
        self.conv3 = RB2(32,32)
        
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.fc1 = nn.Linear(32, 14)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        out = self.conv3(x)
        
        out = self.avgpool(out)
        out1 = out.reshape(x.shape[0], -1)
        out = self.fc1(out1)

        return out1,out    

In [4]:
from torchsummary import summary
model = MAPB()
model = model.to('cuda')
summary(model,input_size=(1,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 64, 64]              80
       BatchNorm2d-2            [-1, 8, 64, 64]              16
             PReLU-3            [-1, 8, 64, 64]               1
            Conv2d-4            [-1, 8, 64, 64]              80
       BatchNorm2d-5            [-1, 8, 64, 64]              16
             PReLU-6            [-1, 8, 64, 64]               1
            Conv2d-7            [-1, 8, 64, 64]              80
       BatchNorm2d-8            [-1, 8, 64, 64]              16
             PReLU-9            [-1, 8, 64, 64]               1
           Conv2d-10            [-1, 8, 64, 64]              80
      BatchNorm2d-11            [-1, 8, 64, 64]              16
            PReLU-12            [-1, 8, 64, 64]               1
AdaptiveAvgPool2d-13             [-1, 32, 1, 1]               0
           Conv2d-14             [-1, 1

In [5]:
net=model

In [6]:
net.load_state_dict(torch.load(r"C:\Users\cccc\Desktop\MAPNET2\weight.pth"))

<All keys matched successfully>

In [7]:
import joblib
categories = list(range(0, 14))
weibull_model = joblib.load('weibull_model.pkl')

In [8]:
X_test1 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\X_TEST1.npy")
X_test1.shape
X_test1=X_test1.reshape(len(X_test1),1,64,64)
X_test1.shape

X_test2 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\X_TEST2.npy")
X_test2.shape
X_test2=X_test2.reshape(len(X_test2),1,64,64)
X_test2.shape

X_test3 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\X_TEST3.npy")
X_test3.shape
X_test3=X_test3.reshape(len(X_test3),1,64,64)
X_test3.shape

(4800, 1, 64, 64)

In [9]:
X_test1=np.concatenate((X_test1,X_test2,X_test3),axis=0)

In [10]:
X_test1.shape

(14400, 1, 64, 64)

In [11]:


#X_test1=X_test.reshape(14400,1,224,224)
import numpy as np
import os
#label the results
y1 = np.zeros(800)
y2 = np.ones(800)*1
y3 = np.ones(800)*2
y4 = np.ones(800)*3
y5 = np.ones(800)*4
y6 = np.ones(800)*5
y7 = np.ones(800)*6
y8 = np.ones(800)*7
y9 =  np.ones(800)*8
y10 = np.ones(800)*9 
y11 = np.ones(800)*10 
y12 = np.ones(800)*11 
y13 = np.ones(800)*12 
y14 = np.ones(800)*13 
y15 = np.ones(800*4)*14
yy= np.concatenate((y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15),axis=0)
#yy= yy.reshape(-1,1)
print(yy.shape)

y = yy
#y = y.reshape(-1,1)
print(y.shape)
ytest1=y

(14400,)
(14400,)


In [12]:
## CLOSED RECOGNITION
net.eval()
total = 0
acc=np.zeros([14,8])
for num_s in range (14):
    for num_SNR in range (8):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 


        pre=[]
        
        for score in scores:
            ss = np.argmax(score)
            pre.append(ss)
            
        pre = torch.from_numpy(np.array(pre))

        pred= torch.from_numpy(np.array(pre))
        labels=torch.from_numpy(np.array(labels))
        correct += pred.eq(labels).sum().item()


        acc[num_s,num_SNR]=correct/100
np.sum(acc,0)/14     

array([1.        , 1.        , 1.        , 0.99928571, 1.        ,
       0.99785714, 0.99214286, 0.97142857])

In [13]:
## OPEN RECOGNITION

In [14]:
import numpy as np
import scipy.spatial.distance as spd
import torch
import libmr


def calc_distance(query_score, mcv, eu_weight, distance_type='euclidean'):
    if distance_type == 'euclidean':
        query_distance = spd.euclidean(mcv, query_score)
    elif distance_type == 'cosine':
        query_distance = spd.cosine(mcv, query_score)
    else:
        print("distance type not known: enter either of euclidean or cosine")
    return query_distance


def fit_weibull(means, dists, categories, tailsize=20, distance_type='euclidean'):
    weibull_model = {}
    for mean, dist, category_name in zip(means, dists, categories):
        weibull_model[category_name] = {}
        weibull_model[category_name]['distances_{}'.format(distance_type)] = dist[distance_type]
        weibull_model[category_name]['mean_vec'] = mean
        weibull_model[category_name]['weibull_model'] = []
        for channel in range(mean.shape[0]):
            mr = libmr.MR()
            tailtofit = np.sort(dist[distance_type][channel, :])[-tailsize:]
            mr.fit_high(tailtofit, len(tailtofit))
            weibull_model[category_name]['weibull_model'].append(mr)
    return weibull_model


def query_weibull(category_name, weibull_model, distance_type='euclidean'):
    return [weibull_model[category_name]['mean_vec'],
            weibull_model[category_name]['distances_{}'.format(distance_type)],
            weibull_model[category_name]['weibull_model']]

def compute_pro_un_prob(scores, scores_u):
    prob_scores, prob_unknowns = [], []
    for s, su in zip(scores, scores_u):
        channel_scores = np.exp(s)
        channel_unknown = np.exp(np.sum(su))

        total_denom = np.sum(channel_scores) + channel_unknown
        prob_scores.append(channel_scores / total_denom)
        prob_unknowns.append(channel_unknown / total_denom)

    # Take channel mean
    scores = np.mean(prob_scores, axis=0)
    unknowns = np.mean(prob_unknowns, axis=0)
    modified_scores = scores.tolist() + [unknowns]
    return modified_scores


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()


def pro_un(weibull_model, categories, input_score, eu_weight, alpha=10, distance_type='euclidean'):
    nb_classes = len(categories)

    ranked_list = input_score.argsort().ravel()[::-1][:alpha]
    alpha_weights = [((alpha + 1) - i) / float(alpha) for i in range(1, alpha + 1)]
    omega = np.zeros(nb_classes)
    omega[ranked_list] = alpha_weights

    scores, scores_u = [], []
    for channel, input_score_channel in enumerate(input_score):
        score_channel, score_channel_u = [], []
        for c, category_name in enumerate(categories):
            mav, dist, model = query_weibull(category_name, weibull_model, distance_type)
            channel_dist = calc_distance(input_score_channel, mav[channel], eu_weight, distance_type)
            wscore = model[channel].w_score(channel_dist)
            modified_score = input_score_channel[c] * (1 - wscore * omega[c])
            score_channel.append(modified_score)
            score_channel_u.append(input_score_channel[c] - modified_score)

        scores.append(score_channel)
        scores_u.append(score_channel_u)

    scores = np.asarray(scores)
    scores_u = np.asarray(scores_u)

    pro_un_prob = np.array(compute_pro_un_prob(scores, scores_u))
    softmax_prob = softmax(np.array(input_score.ravel()))
    return pro_un_prob, softmax_prob


def compute_channel_distances(mavs, features, eu_weight=0.5):

    eucos_dists, eu_dists, cos_dists = [], [], []
    for channel, mcv in enumerate(mavs):  
        eu_dists.append([spd.euclidean(mcv, feat[channel]) for feat in features])
        cos_dists.append([spd.cosine(mcv, feat[channel]) for feat in features])
        eucos_dists.append([spd.euclidean(mcv, feat[channel]) * eu_weight +
                            spd.cosine(mcv, feat[channel]) for feat in features])

    return {'eucos': np.array(eucos_dists), 'cosine': np.array(cos_dists), 'euclidean': np.array(eu_dists)}


def compute_train_score_and_mavs_and_dists(train_class_num,trainloader,device,net):
    scores = [[] for _ in range(train_class_num)]
    with torch.no_grad():

        for batch_idx, batch in enumerate(trainloader):
            inputs,targets = batch[0], batch[1]
            inputs  = inputs.cuda()
            targets  = targets.cuda()
            targets=targets.long()   
            

            embed_fea1,outputs = net(inputs)
            for score, t in zip(outputs, targets):
                if torch.argmax(score) == t:
                    scores[t].append(score.unsqueeze(dim=0).unsqueeze(dim=0))
    scores = [torch.cat(x).cpu().numpy() for x in scores]  # (N_c, 1, C) * C
    mavs = np.array([np.mean(x, axis=0) for x in scores])  # (C, 1, C)
    dists = [compute_channel_distances(mcv, score) for mcv, score in zip(mavs, scores)]
    return scores, mavs, dists


In [15]:
def Recalibrate(weibull_model, categories, input_score, eu_weight, alpha=10, distance_type='euclidean'):

    nb_classes = len(categories)

    ranked_list = input_score.argsort().ravel()[::-1][:alpha]
    alpha_weights = [((alpha + 1) - i) / float(alpha) for i in range(1, alpha + 1)]
    omega = np.zeros(nb_classes)
    omega[ranked_list] = alpha_weights

    scores, scores_u = [], []
    for channel, input_score_channel in enumerate(input_score):
        score_channel, score_channel_u = [], []
        for c, category_name in enumerate(categories):
            mav, dist, model = query_weibull(category_name, weibull_model, distance_type)
            channel_dist = calc_distance(input_score_channel, mav[channel], eu_weight, distance_type)
            wscore = model[channel].w_score(channel_dist)
            modified_score = input_score_channel[c] * (1 - wscore * omega[c])
            score_channel.append(modified_score)
            score_channel_u.append(input_score_channel[c] - modified_score)

        scores.append(score_channel)
        scores_u.append(score_channel_u)

    scores = np.asarray(scores)
    scores_u = np.asarray(scores_u)

    Recalibrate_prob = np.array(compute_Recalibrate_prob(scores, scores_u))
    softmax_prob = softmax(np.array(input_score.ravel()))
    return Recalibrate_prob, softmax_prob

def compute_Recalibrate_prob(scores, scores_u):
    prob_scores, prob_unknowns = [], []
    for s, su in zip(scores, scores_u):
        channel_scores = np.exp(s)
        channel_unknown = np.exp(np.sum(su))

        total_denom = np.sum(channel_scores) + channel_unknown
        prob_scores.append(channel_scores / total_denom)
        prob_unknowns.append(channel_unknown / total_denom)

    # Take channel mean
    scores = np.mean(prob_scores, axis=0)
    unknowns = np.mean(prob_unknowns, axis=0)
    modified_scores = scores.tolist() + [unknowns]
    return modified_scores

In [16]:
net.eval()

total = 0
acc=np.zeros([18,8])
acc2=np.zeros([18,8])
acc3=np.zeros([18,8])
for num_s in range (18):
    for num_SNR in range (8):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 

        pred_Recalibrate = [] 
        score_Recalibrate = []
        
        for score in scores:
            so, ss = Recalibrate(weibull_model, categories, score,
                             0.5, 3, "euclidean")
            pred_Recalibrate.append(np.argmax(so))
            score_Recalibrate.append(so)        
        pred_Recalibrate = torch.from_numpy(np.array(pred_Recalibrate))
        
        labels=torch.from_numpy(np.array(labels))
        correct += pred_Recalibrate.eq(labels).sum().item()

        acc[num_s,num_SNR]=correct/100

In [17]:
acc

array([[1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.97],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 0.85],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.98],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 0.92],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.97],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.98, 0.89],
       [1.  , 1.  , 1.  , 1.  , 0.98, 0.96, 0.93, 0.82],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.92],
       [1.  , 1.  , 1.  , 1.  , 1.  , 0.97, 0.9 , 0.66],
       [1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 0.92, 0.81],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.95],
       [1.  , 1.  , 1.  , 0.99, 0.99, 0.95, 0.9 , 0.82],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 0.86],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.99, 0.98],
       [1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 1.  , 0.96],
       [1.  , 1.  , 1.  , 0.99, 1.  , 0.99, 1.  , 1.  ],
       [1.  , 1.  , 1.  , 1.  , 0.99, 0.98, 0.97, 0.93],
       [1.  , 1.  , 1.  , 1.  ,

In [18]:
np.mean(acc[0:14],0)

array([1.        , 1.        , 1.        , 0.99928571, 0.99785714,
       0.99071429, 0.97071429, 0.88571429])

In [19]:
np.mean(acc[14:18],0)

array([1.    , 1.    , 1.    , 0.9975, 0.995 , 0.99  , 0.99  , 0.9725])

In [20]:
np.mean(np.mean(acc[0:14],0))

0.9805357142857143

In [21]:
np.mean(np.mean(acc[14:18],0))

0.993125

In [22]:
#Measured

In [23]:
X_test1 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\SFM.npy")
X_test1.shape
LE=len(X_test1)

net.eval()
X_test1=X_test1.reshape([LE,1, 64,64])
total = 0
acc=np.zeros([16,8])
acc2=np.zeros([16,8])
acc3=np.zeros([16,8])
for num_s in range (1):
    for num_SNR in range (1):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1).float().cuda()
        #targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        #targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 

        pred_Recalibrate = []
        score_Recalibrate = []
        
        for score in scores:
            so, ss = Recalibrate(weibull_model, categories, score,
                             0.5, 3, "euclidean")
            pred_Recalibrate.append(np.argmax(so))
            score_Recalibrate.append(so)        
        pred_Recalibrate = torch.from_numpy(np.array(pred_Recalibrate))
        

acc=0
for ii in range(100):
    if pred_Recalibrate[ii]==1:
        acc=acc+1
print(acc)   
acc=0
for ii in range(50):
    if pred_Recalibrate[ii]==1:
        acc=acc+1
print(acc)  
acc=0
for ii in range(50,100):
    if pred_Recalibrate[ii]==1:
        acc=acc+1
print(acc)  

90
50
40


In [24]:
X_test1 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\NS.npy")
X_test1.shape
LE=len(X_test1)

net.eval()
X_test1=X_test1.reshape([LE,1, 64,64])
total = 0
acc=np.zeros([16,8])
acc2=np.zeros([16,8])
acc3=np.zeros([16,8])
for num_s in range (1):
    for num_SNR in range (1):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1).float().cuda()
        #targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        #targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 

        pred_Recalibrate = []
        score_Recalibrate = []
        
        for score in scores:
            so, ss = Recalibrate(weibull_model, categories, score,
                             0.5, 3, "euclidean")
            pred_Recalibrate.append(np.argmax(so))
            score_Recalibrate.append(so)        
        pred_Recalibrate = torch.from_numpy(np.array(pred_Recalibrate))
        

acc=0
for ii in range(100):
    if pred_Recalibrate[ii]==4:
        acc=acc+1
print(acc)   
acc=0
for ii in range(50):
    if pred_Recalibrate[ii]==4:
        acc=acc+1
print(acc)  
acc=0
for ii in range(50,100):
    if pred_Recalibrate[ii]==4:
        acc=acc+1
print(acc)  

98
50
48


In [25]:
X_test1 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\LFM.npy")
X_test1.shape
LE=len(X_test1)

net.eval()
X_test1=X_test1.reshape([LE,1, 64,64])
total = 0
acc=np.zeros([16,8])
acc2=np.zeros([16,8])
acc3=np.zeros([16,8])
for num_s in range (1):
    for num_SNR in range (1):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1).float().cuda()
        #targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        #targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 

        pred_Recalibrate = []
        score_Recalibrate = []
        
        for score in scores:
            so, ss = Recalibrate(weibull_model, categories, score,
                             0.5, 3, "euclidean")
            pred_Recalibrate.append(np.argmax(so))
            score_Recalibrate.append(so)        
        pred_Recalibrate = torch.from_numpy(np.array(pred_Recalibrate))
        

acc=0
for ii in range(100):
    if pred_Recalibrate[ii]==0:
        acc=acc+1
print(acc)   
acc=0
for ii in range(50):
    if pred_Recalibrate[ii]==0:
        acc=acc+1
print(acc)  
acc=0
for ii in range(50,100):
    if pred_Recalibrate[ii]==0:
        acc=acc+1
print(acc)  

84
50
34


In [26]:
X_test1 =np.load(r"C:\Users\cccc\Desktop\MAPNET2\TFI\UN.npy")
X_test1.shape
LE=len(X_test1)

net.eval()
X_test1=X_test1.reshape([LE,1, 64,64])
total = 0
acc=np.zeros([16,8])
acc2=np.zeros([16,8])
acc3=np.zeros([16,8])
for num_s in range (1):
    for num_SNR in range (1):
        correct = 0
        correct2 = 0
        correct3 = 0
        scores, labels = [], []

        inputs=torch.from_numpy(X_test1).float().cuda()
        #targets=torch.from_numpy(ytest1[num_s*800+num_SNR*100+0:num_s*800+num_SNR*100+100]).float().cuda()
        #targets=targets.long()
        _,outputs = net(inputs)
        scores.append(outputs)
        labels.append(targets)
        
        scores = torch.cat(scores,dim=0).cpu().detach().numpy()
        labels = torch.cat(labels,dim=0).cpu().detach().numpy()     
        
        
        
        scores = np.array(scores)[:, np.newaxis, :]
        labels = np.array(labels) 

        pred_Recalibrate = []
        score_Recalibrate = []
        
        for score in scores:
            so, ss = Recalibrate(weibull_model, categories, score,
                             0.5, 3, "euclidean")
            pred_Recalibrate.append(np.argmax(so))
            score_Recalibrate.append(so)        
        pred_Recalibrate = torch.from_numpy(np.array(pred_Recalibrate))
        

acc=0
for ii in range(100):
    if pred_Recalibrate[ii]==14:
        acc=acc+1
print(acc)   

100
